In [1]:
import torch
import numpy as np

print(f"Is GPU available? {torch.cuda.is_available()}")
print(f"PyTorch version: {torch.__version__}")

Is GPU available? True
PyTorch version: 2.8.0+cu128


### Tensors

In [2]:
tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1, 2, 3])
tensor2d = torch.tensor([[1, 2],
                         [3, 4]])
tensor3d = torch.tensor([[[1, 2], [3, 4]],
                         [[5, 6], [7, 8]]])

nptensor = np.array([[1, 2], [3, 4]])
tensorfromnumpy = torch.from_numpy(nptensor)

print(tensorfromnumpy)

print("Number of tensors' dimensions:")
print(tensor0d.ndim)
print(tensor1d.ndim)
print(tensor2d.ndim)
print(tensor3d.ndim)

tensor([[1, 2],
        [3, 4]], dtype=torch.int32)
Number of tensors' dimensions:
0
1
2
3


In [3]:
floatvec = torch.tensor([1.0, 2.0, 3.0])
print("Tensors datatypes:")
print(floatvec.dtype)
print(floatvec.to(torch.int64).dtype)

Tensors datatypes:
torch.float32
torch.int64


In [6]:
print(tensorfromnumpy.shape)
floatvec.shape

torch.Size([2, 2])


torch.Size([3])

In [10]:
tensor = torch.tensor([[1, 2, 3],
                        [4, 5, 6]])
print(tensor)
tensor_reshaped = tensor.reshape(3, 2)
print(tensor_reshaped)
print(tensor.view(3, 2))

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [12]:
print(torch.matmul(tensor, tensor.T))
print(tensor @ tensor.T)

tensor([[14, 32],
        [32, 77]])
tensor([[14, 32],
        [32, 77]])
